This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from mordred import Calculator, descriptors

import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
# class CFG:
#     TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
#     SEED = 42
#     FOLDS = 10

In [4]:
# useless_cols = [   
    
#     'MaxPartialCharge', 
#     # Nan data
#     'BCUT2D_MWHI',
#     'BCUT2D_MWLOW',
#     'BCUT2D_CHGHI',
#     'BCUT2D_CHGLO',
#     'BCUT2D_LOGPHI',
#     'BCUT2D_LOGPLOW',
#     'BCUT2D_MRHI',
#     'BCUT2D_MRLOW',

#     # Constant data
#     'NumRadicalElectrons',
#     'SMR_VSA8',
#     'SlogP_VSA9',
#     'fr_barbitur',
#     'fr_benzodiazepine',
#     'fr_dihydropyridine',
#     'fr_epoxide',
#     'fr_isothiocyan',
#     'fr_lactam',
#     'fr_nitroso',
#     'fr_prisulfonamd',
#     'fr_thiocyan',

#     # High correlated data >0.95
#     'MaxEStateIndex',
#     'HeavyAtomMolWt',
#     'ExactMolWt',
#     'NumValenceElectrons',
#     'Chi0',
#     'Chi0n',
#     'Chi0v',
#     'Chi1',
#     'Chi1n',
#     'Chi1v',
#     'Chi2n',
#     'Kappa1',
#     'LabuteASA',
#     'HeavyAtomCount',
#     'MolMR',
#     'Chi3n',
#     'BertzCT',
#     'Chi2v',
#     'Chi4n',
#     'HallKierAlpha',
#     'Chi3v',
#     'Chi4v',
#     'MinAbsPartialCharge',
#     'MinPartialCharge',
#     'MaxAbsPartialCharge',
#     'FpDensityMorgan2',
#     'FpDensityMorgan3',
#     'Phi',
#     'Kappa3',
#     'fr_nitrile',
#     'SlogP_VSA6',
#     'NumAromaticCarbocycles',
#     'NumAromaticRings',
#     'fr_benzene',
#     'VSA_EState6',
#     'NOCount',
#     'fr_C_O',
#     'fr_C_O_noCOO',
#     'NumHDonors',
#     'fr_amide',
#     'fr_Nhpyrrole',
#     'fr_phenol',
#     'fr_phenol_noOrthoHbond',
#     'fr_COO2',
#     'fr_halogen',
#     'fr_diazo',
#     'fr_nitro_arom',
#     'fr_phos_ester'
# ]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id']

In [6]:
mols_test = [Chem.MolFromSmiles(s) for s in test.SMILES]

# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_test = calc.pandas(mols_test)
desc_test.head()

100%|██████████| 3/3 [00:00<00:00,  4.63it/s]


ABC  \
0  module 'numpy' has no attribute 'float'.\n`np....   
1  module 'numpy' has no attribute 'float'.\n`np....   
2  module 'numpy' has no attribute 'float'.\n`np....   

                                               ABCGG  nAcid  nBase    SpAbs_A  \
0  module 'numpy' has no attribute 'float'.\n`np....      0      0  51.369068   
1  module 'numpy' has no attribute 'float'.\n`np....      0      0  53.698919   
2  module 'numpy' has no attribute 'float'.\n`np....      0      0  60.526875   

    SpMax_A  SpDiam_A     SpAD_A   SpMAD_A   LogEE_A     VE1_A     VE2_A  \
0  2.608687  5.217373  51.369068  1.252904  4.633213  4.054325  0.098886   
1  2.429583  4.859165  53.698919  1.309730  4.643161  4.685471  0.114280   
2  2.564154  4.983326  60.526875  1.315802  4.764144  4.942681  0.107450   

      VE3_A          VR1_A         VR2_A      VR3_A  nAromAtom  nAromBond  \
0  2.810771  138187.404971   3370.424511  13.247353         24         24   
1  2.955453     981.187389     23.931400   8.299750         30         30   
2  3.123964  637481.849542  13858.301077  14.891337         24         24   

   nAtom  nHeavyAtom  nSpiro  nBridgehead  nHetero  nH  nB  nC  nN  nO  nS  \
0     59          41       0            0       12  18   0  29   2   2   0   
1     67          41       0            0        6  26   0  35   0   4   0   
2     76          46       0            0       10  30   0  36   2   6   0   

   nP  nF  nCl  nBr  nI  nX  ATS0dv  ATS1dv  ATS2dv  ATS3dv  ATS4dv  ATS5dv  \
0   0   6    0    0   0   6   754.0   647.0  1100.0  1133.0  1481.0   988.0   
1   0   0    0    0   0   0   534.0   554.0   760.0   789.0   700.0   672.0   
2   0   0    0    0   0   0   648.0   655.0   974.0  1023.0   953.0   805.0   

   ATS6dv  ATS7dv  ATS8dv  ATS0d  ATS1d  ATS2d  ATS3d  ATS4d  ATS5d  ATS6d  \
0   788.0   662.0   564.0  234.0  291.0  429.0  490.0  466.0  427.0  353.0   
1   648.0   576.0   550.0  244.0  303.0  463.0  534.0  512.0  485.0  465.0   
2   797.0   722.0   743.0  276.0  355.0  550.0  599.0  573.0  552.0  534.0   

   ATS7d  ATS8d                                            ATS0s  \
0  311.0  294.0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  414.0  361.0  missing intrinsic state for ['*'] (ATS0s/Props)   
2  515.0  470.0  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS1s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS2s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS3s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS4s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS5s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS6s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS7s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS8s   ATS0Z   ATS1Z   ATS2Z  \
0  missing intrinsic state for ['*'] (ATS0s/Props)  1774.0  1789.0  2888

In [7]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv',      low_memory=False)
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',      low_memory=False)
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
ds1_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv',      low_memory=False)
ds2_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv',      low_memory=False)
ds3_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv',      low_memory=False)
ds4_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',      low_memory=False)

tg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',      low_memory=False)
tc_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',      low_memory=False)
rg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',      low_memory=False)
ffv_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',     low_memory=False)
den_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',      low_memory=False)

### Extra Files

# Preprocessing 

In [8]:
for i in (tg_ds,tc_ds,rg_ds,ffv_ds,den_ds):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)

tg_ds = tg_ds.select_dtypes(exclude=['object', 'category'])
rg_ds = rg_ds.select_dtypes(exclude=['object', 'category'])
ffv_ds = ffv_ds.select_dtypes(exclude=['object', 'category'])
tc_ds = tc_ds.select_dtypes(exclude=['object', 'category'])
den_ds = den_ds.select_dtypes(exclude=['object', 'category'])

In [9]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [10]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
ds1_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)

In [11]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[15:49:21] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[15:49:21] SMILES Parse Error: check for mistakes around position 12:
[15:49:21] *O[Si](*)([R])[R]
[15:49:21] ~~~~~~~~~~~^
[15:49:21] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[15:49:21] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[15:49:21] SMILES Parse Error: check for mistakes around position 28:
[15:49:21] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[15:49:21] ~~~~~~~~~~~~~~~~~~~~^
[15:49:21] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[15:49:21] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[15:49:21] SMILES Parse Error: check for mistakes around position 7:
[15:49:21] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[15:49:21] ~~~~~~^
[15:49:21] SMILES Parse Error: F

In [12]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Feature Extraction 

In [13]:
# SMILESを正規化（canonical化）
# 重複する SMILES を集約して平均 target 値に
# df_train に存在するが欠損している target に、df_extra から値を補完

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')

train = add_extra_data(train, ds1_smiles, 'Tc')
#train = add_extra_data(train, ds2_smiles, 'Density')
train = add_extra_data(train, ds3_smiles, 'Tg')
train = add_extra_data(train, ds4_smiles, 'FFV')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524

For target "Tc" added 0 new samples!
New unique SMILES: 0

For target "Tg" added 0 new samples!
New unique SMILES: 0

For target "FFV" added 862 new samples!
New unique SMILES: 819


In [14]:
# def compute_all_descriptors(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         return [None] * len(desc_names)
#     return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

# def compute_graph_features(smiles, graph_feats):
#     mol = Chem.MolFromSmiles(smiles)
#     adj = rdmolops.GetAdjacencyMatrix(mol)
#     G = nx.from_numpy_array(adj)

#     graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
#     graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
#     graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

# train = pd.concat([train, preprocessing(train)], axis=1)
# test = pd.concat([test, preprocessing(test)], axis=1)

# # Find constant columns for each target
# all_features = train.columns[7:].tolist()
# features = {}
# for target in CFG.TARGETS:
#     const_descs = []
#     for col in train.columns.drop(CFG.TARGETS):
#         if train[train[target].notnull()][col].nunique() == 1:
#             const_descs.append(col)
#     features[target] = [f for f in all_features if f not in const_descs]

# print(train.shape)
# train['Ipc']=np.log10(train['Ipc'])  
# for n in train.columns[7:]:
#     train[n]=train[n].replace(-np.inf,np.nan)
#     train[n]=train[n].replace(np.inf,np.nan)    
#     train[n].fillna(train[n].mean())
  
# print(train.shape)
# test['Ipc']=np.log10(test['Ipc'])
# for n in test.columns[7:]:
#     train[n]=train[n].replace(-np.inf,np.nan)
#     train[n]=train[n].replace(np.inf,np.nan)      
#     test[n].fillna(train[n].mean())

In [15]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,0.932,NaN
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416.0,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187.0,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
...,...,...,...,...,...,...,...
10075,NaN,*c1cccc(OCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5...,NaN,0.349095,NaN,NaN,NaN
10076,NaN,*c1cccc(OCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5C...,NaN,0.350892,NaN,NaN,NaN
10077,NaN,*c1cccc(OCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5C(...,NaN,0.345386,NaN,NaN,NaN
10078,NaN,*c1cccc(Oc2cccc(Oc3cccc(N4C(=O)c5ccc(Oc6ccc(Sc...,NaN,0.362224,NaN,NaN,NaN


# Data Preparation For Model Training 

In [16]:
# We'll separate train to be one model for each target variable.
t_1=train[['SMILES','Tg']].copy()
t_2=train[['SMILES','FFV']].copy()
t_3=train[['SMILES','Tc']].copy()
t_4=train[['SMILES','Density']].copy()
t_5=train[['SMILES','Rg']].copy()

# We will drop the rows with missing values related to that target after separation.
#This is important , dropping them beforehand would result Null for all data.
t_1.dropna(inplace=True)
t_2.dropna(inplace=True)
t_3.dropna(inplace=True)
t_4.dropna(inplace=True)
t_5.dropna(inplace=True)

t_1.index = range(0, len(t_1))
t_2.index = range(0, len(t_2))
t_3.index = range(0, len(t_3))
t_4.index = range(0, len(t_4))
t_5.index = range(0, len(t_5))

In [17]:
mols_test_t1 = [Chem.MolFromSmiles(s) for s in t_1.SMILES]
mols_test_t2 = [Chem.MolFromSmiles(s) for s in t_2.SMILES]
mols_test_t3 = [Chem.MolFromSmiles(s) for s in t_3.SMILES]
mols_test_t4 = [Chem.MolFromSmiles(s) for s in t_4.SMILES]
mols_test_t5 = [Chem.MolFromSmiles(s) for s in t_5.SMILES]
# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_t1 = calc.pandas(mols_test_t1)
desc_t2 = calc.pandas(mols_test_t1)
desc_t3 = calc.pandas(mols_test_t1)
desc_t4 = calc.pandas(mols_test_t1)
desc_t5 = calc.pandas(mols_test_t1)

100%|██████████| 1161/1161 [02:34<00:00,  7.50it/s]


In [18]:
desc_t1['Tg']=t_1['Tg'].copy()
desc_t2['FFV']=t_2['FFV'].copy()
desc_t3['Tc']=t_3['Tc'].copy()
desc_t4['Density']=t_4['Density'].copy()
desc_t5['Rg']=t_5['Rg'].copy()

In [19]:
desc_t1 = desc_t1.select_dtypes(exclude=['object', 'category'])
desc_t2 = desc_t2.select_dtypes(exclude=['object', 'category'])
desc_t3 = desc_t3.select_dtypes(exclude=['object', 'category'])
desc_t4 = desc_t4.select_dtypes(exclude=['object', 'category'])
desc_t5 = desc_t5.select_dtypes(exclude=['object', 'category'])

tg_ds=pd.concat([tg_ds, desc_t1], ignore_index=True)
ffv_ds=pd.concat([ffv_ds, desc_t2], ignore_index=True)
tc_ds=pd.concat([tc_ds, desc_t3], ignore_index=True)
den_ds=pd.concat([den_ds, desc_t4], ignore_index=True)
rg_ds=pd.concat([rg_ds, desc_t5], ignore_index=True)
# tg=t_1.merge(train,on='SMILES',how='left')
# ffv=t_2.merge(train,on='SMILES',how='left')
# tc=t_3.merge(train,on='SMILES',how='left')
# density=t_4.merge(train,on='SMILES',how='left')
# rg=t_5.merge(train,on='SMILES',how='left')

# Model 

In [20]:

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    train_cols = set(train_d.columns) - {target}
    test_cols = set(test_d.columns)
   # Intersect the feature columns
    common_cols = list(train_cols & test_cols)
    X=train_d[common_cols].copy()
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d[common_cols].copy())
       return submission

# **Model Evaluation

In [21]:

models_to_compare = {
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
}

In [22]:
scaler = StandardScaler()

for name, model_init in models_to_compare.items():
    try:
        score = model(tg_ds, desc_test, model_init, 'Tg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 23.22152
CatBoost             : MAE = 24.63709


In [23]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(ffv_ds, desc_test, model_cls, 'FFV', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.00883
CatBoost             : MAE = 0.00815


In [24]:
tc_ds = tc_ds.dropna(subset=['Tc'])
for name, model_cls in models_to_compare.items():
    try:
        score = model(tc_ds, desc_test, model_cls, 'Tc', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.06419
CatBoost             : MAE = 0.06007


In [25]:

for name, model_cls in models_to_compare.items():
    try:
        score = model(den_ds, desc_test, model_cls, 'Density', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.11613
CatBoost             : MAE = 0.09879


In [26]:
rg_ds = rg_ds.dropna(subset=['Rg'])
for name, model_cls in models_to_compare.items():
    try:
        score = model(rg_ds, desc_test, model_cls, 'Rg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 3.14281
CatBoost             : MAE = 2.83314


# Final Model For Submission

In [27]:

'''
11297.2s	273	=== Best models summary ===
11297.2s	274	Tg: CatBoost  CV_MAE=42.369263  params={'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
11297.2s	275	FFV: XGBoost  CV_MAE=0.006229  params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.0, 'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
11297.2s	276	Tc: LightGBM  CV_MAE=0.032098  params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 500, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
11297.2s	277	Density: GradientBoosting  CV_MAE=0.065598  params={'subsample': 0.8, 'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
11297.2s	278	Rg: GradientBoosting  CV_MAE=1.717341  params={'subsample': 0.8, 'n_estimators': 1000, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.01}
'''

sub={'id':ID,'Tg':model(tg_ds,desc_test,XGBRegressor,'Tg',submission=True),
     'FFV':model(ffv_ds,desc_test,CatBoostRegressor,'FFV',submission=True),
     'Tc':model(tc_ds,desc_test,CatBoostRegressor,'Tc',submission=True),
     'Density':model(den_ds,desc_test,CatBoostRegressor,'Density',submission=True),
     'Rg':model(rg_ds,desc_test,CatBoostRegressor,'Rg',submission=True)
     }

Learning rate set to 0.058003
0:	learn: 0.0286480	total: 97ms	remaining: 1m 36s
1:	learn: 0.0280704	total: 158ms	remaining: 1m 18s
2:	learn: 0.0275911	total: 221ms	remaining: 1m 13s
3:	learn: 0.0270972	total: 284ms	remaining: 1m 10s
4:	learn: 0.0266865	total: 347ms	remaining: 1m 9s
5:	learn: 0.0262247	total: 408ms	remaining: 1m 7s
6:	learn: 0.0258368	total: 470ms	remaining: 1m 6s
7:	learn: 0.0254808	total: 530ms	remaining: 1m 5s
8:	learn: 0.0251761	total: 590ms	remaining: 1m 5s
9:	learn: 0.0248450	total: 652ms	remaining: 1m 4s
10:	learn: 0.0244853	total: 715ms	remaining: 1m 4s
11:	learn: 0.0241498	total: 777ms	remaining: 1m 3s
12:	learn: 0.0238784	total: 844ms	remaining: 1m 4s
13:	learn: 0.0236030	total: 904ms	remaining: 1m 3s
14:	learn: 0.0233200	total: 965ms	remaining: 1m 3s
15:	learn: 0.0230964	total: 1.05s	remaining: 1m 4s
16:	learn: 0.0228838	total: 1.14s	remaining: 1m 5s
17:	learn: 0.0226829	total: 1.21s	remaining: 1m 5s
18:	learn: 0.0224674	total: 1.27s	remaining: 1m 5s
19:	lear

In [28]:
submission=pd.DataFrame(sub)

In [29]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,154.518600,0.367060,0.196444,0.950228,18.060502
1,1422188626,151.577301,0.365717,0.250793,0.925384,15.980734
2,2032016830,123.598145,0.364041,0.267336,0.963891,16.706739


In [30]:
submission.to_csv('submission.csv',index=False)